In [ ]:
# https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np

import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [ ]:
def char_split(input_string):
    # Splits each string into individual characters
    return tf.strings.unicode_split(input_string, 'UTF-8')

data_file = "transcription.txt"  # Adjust path as needed

with open(data_file, encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

text_pairs = []
for line in lines:
    # Each line is: word<TAB>transcription
    source, target = line.split("\t")
    # We add start/end markers around target
    target = "@" + target + "#"
    text_pairs.append((source, target))

random.shuffle(text_pairs)

print("Total pairs:", len(text_pairs))

# Split into train, val, test (for example 80/10/10)
num_total = len(text_pairs)
num_val = int(0.1 * num_total)
num_test = int(0.1 * num_total)
num_train = num_total - num_val - num_test

train_pairs = text_pairs[:num_train]
val_pairs = text_pairs[num_train:num_train+num_val]
test_pairs = text_pairs[num_train+num_val:]


Total pairs: 130416


In [ ]:
vocab_size = 200  # Adjust to fit your data (number of unique chars + special tokens)
sequence_length = 32  # Max length in chars for source
target_sequence_length = 32  # Max length in chars for target + 1
batch_size = 64

src_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=None,  # no lowercasing, etc.
    split=char_split,  # custom char-level split
)

tgt_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=target_sequence_length + 1,  # +1 for offset
    standardize=None,
    split=char_split,
)

train_src_texts = [pair[0] for pair in train_pairs]
train_tgt_texts = [pair[1] for pair in train_pairs]

# Adapt the vectorizers on training data only
src_vectorization.adapt(train_src_texts)
tgt_vectorization.adapt(train_tgt_texts)

print("Source char vocab size:", len(src_vectorization.get_vocabulary()))
print("Target char vocab size:", len(tgt_vectorization.get_vocabulary()))


Source char vocab size: 51
Target char vocab size: 45


In [ ]:
def format_dataset(src, tgt):
    src = src_vectorization(src)
    tgt = tgt_vectorization(tgt)
    return (
        {
            "encoder_inputs": src,
            "decoder_inputs": tgt[:, :-1],
        },
        tgt[:, 1:],  # shift by 1
    )

def make_dataset(pairs):
    src_texts, tgt_texts = zip(*pairs)
    src_texts = list(src_texts)
    tgt_texts = list(tgt_texts)
    dataset = tf_data.Dataset.from_tensor_slices((src_texts, tgt_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(1024).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
test_ds = make_dataset(test_pairs)

for inputs, targets in train_ds.take(1):
    print("encoder_inputs shape:", inputs["encoder_inputs"].shape)
    print("decoder_inputs shape:", inputs["decoder_inputs"].shape)
    print("targets shape:", targets.shape)


encoder_inputs shape: (64, 32)
decoder_inputs shape: (64, 32)
targets shape: (64, 32)


In [5]:
##############################################################################
# The original Transformer code (unchanged) starts here
##############################################################################
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            # We build a padding mask for MultiHeadAttention.
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        inputs, encoder_outputs = inputs
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, sequence_length, sequence_length))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config
##############################################################################
# The original Transformer code (unchanged) ends here
##############################################################################


In [6]:
embed_dim = 128
latent_dim = 512
num_heads = 4

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.3)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)


In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"],
)
transformer.summary()


Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_12   │ (None, None, 128)      │         29,696 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_14 (NotEqual)   │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_13   │ (None, None, 128)      │         29,696 │ decoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_3     │ (None, None, 128)      │        396,032 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │ not_equal_14[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_15 (NotEqual)   │ (None, None)           │              0 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_3     │ (None, None, 128)      │        660,096 │ positional_embedding_… │
│ (TransformerDecoder)      │                        │                │ transformer_encoder_3… │
│                           │                        │                │ not_equal_15[0][0],    │
│                           │                        │                │ not_equal_14[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_68 (Dropout)      │ (None, None, 128)      │              0 │ transformer_decoder_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_86 (Dense)          │ (None, None, 200)      │         25,800 │ dropout_68[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,141,320 (4.35 MB)

 Trainable params: 1,141,320 (4.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 30
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 48s 19ms/step - accuracy: 0.1938 - loss: 0.8198 - val_accuracy: 0.2378 - val_loss: 0.1711
Epoch 2/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.2371 - loss: 0.1807 - val_accuracy: 0.2408 - val_loss: 0.1331
Epoch 3/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.2401 - loss: 0.1387 - val_accuracy: 0.2422 - val_loss: 0.1112
Epoch 4/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.2416 - loss: 0.1169 - val_accuracy: 0.2430 - val_loss: 0.1015
Epoch 5/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.2428 - loss: 0.1004 - val_accuracy: 0.2439 - val_loss: 0.0899
Epoch 6/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.2436 - loss: 0.0890 - val_accuracy: 0.2446 - val_loss: 0.0819
Epoch 7/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.2442 - loss: 0.0804 - val_accuracy: 0.2446 - val_loss: 0.0814
Epoch 8/30
1631/1631 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.2446 -

In [ ]:
tgt_vocab = tgt_vectorization.get_vocabulary()
index_to_char = dict(enumerate(tgt_vocab))
max_decoded_length = 30  # max # of predicted characters

def decode_sequence(input_str):
    # Vectorize the source (word)
    tokenized_input = src_vectorization([input_str])
    decoded = "@"
    for i in range(max_decoded_length):
        # Vectorize the partial transcription
        tokenized_target = tgt_vectorization([decoded])[:, :-1]
        predictions = transformer({"encoder_inputs": tokenized_input,
                                   "decoder_inputs": tokenized_target})
        # The predicted character at position i
        next_char_id = ops.convert_to_numpy(ops.argmax(predictions[0, i, :]))
        next_char = index_to_char[int(next_char_id)]

        decoded += next_char
        # Stop if we see the end token
        if next_char == "#":
            break
    return decoded.replace("@", "/").replace("#", "/")

# Try on a random test word
random_word, _ = random.choice(test_pairs)
predicted_transcription = decode_sequence(random_word)
print("Word:", random_word)
print("Predicted transcription:", predicted_transcription)


In [ ]:
transformer.save("char_transformer_model.keras")

In [10]:
transformer = keras.models.load_model(
    "char_transformer_model.keras",
    custom_objects={
        "TransformerEncoder": TransformerEncoder,
        "TransformerDecoder": TransformerDecoder,
        "PositionalEmbedding": PositionalEmbedding
    }
)

In [ ]:
import pickle

# Save src_vectorization vocabulary
src_vocab = src_vectorization.get_vocabulary()
with open("src_vocab.pkl", "wb") as f:
    pickle.dump(src_vocab, f)

# Save tgt_vectorization vocabulary
tgt_vocab = tgt_vectorization.get_vocabulary()
with open("tgt_vocab.pkl", "wb") as f:
    pickle.dump(tgt_vocab, f)

In [3]:
import pickle

def char_split(input_string):
    # Splits each string into individual characters
    return tf.strings.unicode_split(input_string, 'UTF-8')

vocab_size = 200  # Adjust to fit your data (number of unique chars + special tokens)
sequence_length = 32  # Max length in chars for source
target_sequence_length = 32  # Max length in chars for target + 1
batch_size = 64

with open("src_vocab.pkl", "rb") as f:
    loaded_src_vocab = pickle.load(f)

with open("tgt_vocab.pkl", "rb") as f:
    loaded_tgt_vocab = pickle.load(f)

# Recreate the vectorizers
src_vectorization = TextVectorization(
    max_tokens=len(loaded_src_vocab),
    output_mode="int",
    output_sequence_length=32,  # Ensure same length
    standardize=None,
    split=char_split
)
tgt_vectorization = TextVectorization(
    max_tokens=len(loaded_tgt_vocab),
    output_mode="int",
    output_sequence_length=33,  # +1 for shifting
    standardize=None,
    split=char_split
)

# Restore vocabularies
src_vectorization.set_vocabulary(loaded_src_vocab)
tgt_vectorization.set_vocabulary(loaded_tgt_vocab)

# Restore index mappings
index_to_char = {i: char for i, char in enumerate(loaded_tgt_vocab)}

In [11]:
max_decoded_length = 30

def decode_sequence(input_str):
    # Vectorize the source (word)
    tokenized_input = src_vectorization([input_str])
    decoded = "@"
    for i in range(max_decoded_length):
        # Vectorize the partial transcription
        tokenized_target = tgt_vectorization([decoded])[:, :-1]
        predictions = transformer({"encoder_inputs": tokenized_input,
                                   "decoder_inputs": tokenized_target})
        # The predicted character at position i
        next_char_id = ops.convert_to_numpy(ops.argmax(predictions[0, i, :]))
        next_char = index_to_char[int(next_char_id)]

        decoded += next_char
        # Stop if we see the end token
        if next_char == "#":
            break
    return decoded.replace("@", "/").replace("#", "/")

print(decode_sequence("chantourloupettes"))
print(decode_sequence("cyberimposteurs"))
print(decode_sequence("éblistère"))
print(decode_sequence("anticonstitutionnellement"))

/ʃɑ̃tuʁlupɛt/
/sibɛʁɛ̃pɔstœʁ/
/eblistɛʁ/
/ɑ̃tikɔ̃stitysjɔnɛlmɑ̃/


In [13]:
print(decode_sequence("abeille"))

/abɛj/
